In [5]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torchtext.data import get_tokenizer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
from time import time
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
from autocorrect import Speller
from nltk.stem import WordNetLemmatizer
import pycountry
import itertools
import random
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import itertools
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
import importlib


import fonctions_temp_pandas
importlib.reload(fonctions_temp_pandas)
from fonctions_temp_pandas import *
from time import time

tokenizer = get_tokenizer("basic_english")
lemmatizer = WordNetLemmatizer()
embedding_dim = 1000
spell=Speller(lang="en", fast=True)
nltk.download('averaged_perceptron_tagger_eng')


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/martindrieux/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [6]:
def embedding(tweets, keywords=["football", "goal"]):
    feature_matrix = []
    for i, text in enumerate(tweets):
        tokens = tokenizer(text) # ex : 'Hello, I am a test' -> ['hello', 'i', 'am', 'a', 'test']
        token_counts = Counter(tokens)
        len_token = max(len(tokens), 1)
        frequencies = [token_counts.get(keyword, 0)/len_token for keyword in keywords]
        feature_matrix.append(frequencies)
    feature_matrix = np.array(feature_matrix)
    return feature_matrix 

def fit_tfidf_embedding(train_tweets, max_features=2000, ngram_range=(1,1), max_df=0.7, min_df=2, sublinear_tf=False):
    """
    Fit a TF-IDF vectorizer on the training tweets.
    Returns the fitted vectorizer.
    """
    vectorizer = TfidfVectorizer(max_features=max_features,
                                 max_df=max_df,
                                 ngram_range=ngram_range,
                                 min_df=min_df,
                                 sublinear_tf=sublinear_tf)
    vectorizer.fit(train_tweets)
    return vectorizer

def tfidf_embedding(tweets, vectorizer):
    """
    Transform tweets into TF-IDF features using a fitted vectorizer.
    """
    return vectorizer.transform(tweets).toarray()

In [7]:
# Import the preprocessed data
df_train = pd.read_csv("df_train_withoutstemming.csv")
df_eval = pd.read_csv("df_eval_withoutstemming.csv")
# df_train = pd.read_csv("df_train_tweets_with_lemmatization.csv")
# df_eval = pd.read_csv("df_eval_tweets_with_lemmatization.csv")

# str the 'Tweet' column
df_train['Tweet'] = df_train['Tweet'].astype(str)
df_eval['Tweet'] = df_eval['Tweet'].astype(str)
print(f'The shape of the training data is: {df_train.shape}')
print(f'The shape of the evaluation data is: {df_eval.shape}')

# Filter : 
df_train = df_train[df_train["Tweet"].notnull()]  # Remove null values
df_train = df_train[df_train["Tweet"].str.strip() != ""]  # Remove empty strings
df_eval = df_eval[df_eval["Tweet"].notnull()]  # Remove null values
df_eval = df_eval[df_eval["Tweet"].str.strip() != ""]  # Remove empty strings

# # Remove any dupplicates
df_train = df_train.drop_duplicates()
df_eval = df_eval.drop_duplicates()

print(f'The shape of the training data after removing duplicates is: {df_train.shape}')
print(f'The shape of the evaluation data after removing duplicates is: {df_eval.shape}')

df_train_plus = add_freq_normalized(add_time_sec(df_train))
df_eval_plus = add_freq_normalized(add_time_sec(df_eval))

df_train_plus.head()

The shape of the training data is: (1438007, 6)
The shape of the evaluation data is: (1072928, 5)
The shape of the training data after removing duplicates is: (1272324, 6)
The shape of the evaluation data after removing duplicates is: (963395, 5)


,ID,MatchID,PeriodID,EventType,Tweet,Seconds,NormalizedFrequency
0,1_0,1,0,0,sami khedira staing marcelo staing best luck,0,0.607822
1,1_0,1,0,0,damn half ya saying go wasnt even rooting begi...,0,0.607822
2,1_0,1,0,0,ones playing today course fifa couldnt let happen,0,0.607822
3,1_0,1,0,0,injury list neymar reus gundogan gomez l bende...,0,0.607822
4,1_0,1,0,0,still favored vs homefield advantage mt,0,0.607822


In [8]:
# Hyperparameters
output_dim = 1
embedding_dim = 2000
input_dim = embedding_dim +1 # +1 for the normalized frequency
weight_1_0 = 0.55
hidden_dims = [embedding_dim*3,embedding_dim//20, embedding_dim//10]
dropouts = [0.95] * len(hidden_dims)
epochs = 20
lr = 0.001
decay = 1e-5
train= df_train_plus

vectorizer = fit_tfidf_embedding(train['Tweet'], max_features=embedding_dim)

# Transform data
train_X = tfidf_embedding(train['Tweet'], vectorizer)
eval_X = tfidf_embedding(df_eval_plus['Tweet'], vectorizer)

train_X = np.hstack([train_X, train[['NormalizedFrequency']].values.reshape(-1, 1)])
eval_X = np.hstack([eval_X, df_eval_plus[['NormalizedFrequency']].values.reshape(-1, 1)])#

In [ ]:


# Loss and optimizer
criterion = nn.BCELoss()
scheduler = None
print("Testing on SVM with regularization:")
t = time()
svm = SVC(C=0.1, kernel='linear')  # Smaller C increases regularization
svm.fit(train_X, train['EventType'])

print(f"Model trained in {time()-t:.2f} seconds")




Testing on SVM with regularization:


In [ ]:
# Regularized Logistic Regression
print("Testing on Logistic Regression with regularization:")
t = time()
lg = LogisticRegression(max_iter=1000, C=0.1, penalty='l2', solver='liblinear')  # Stronger L2 regularization
lg.fit(train_X, train['EventType'])

print(f"Model trained in {time()-t:.2f} seconds")



In [ ]:
# Regularized Random Forest
print("Testing on Random Forest with regularization:")
t = time()
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=3)  # Depth/leaf regularization
rf.fit(train_X, train['EventType'])

print(f"Model trained in {time()-t:.2f} seconds")


In [ ]:
svm_preds = svm.predict(eval_X)

# Logistic Regression Predictions
lg_preds = lg.predict(eval_X)

# Random Forest Predictions
rf_preds = rf.predict(eval_X)

#create dataframes with tweet ID and predictions
df_eval_svm = pd.DataFrame(df_eval['TweetID'])
df_eval_lg = pd.DataFrame(df_eval['TweetID'])
df_eval_rf = pd.DataFrame(df_eval['TweetID'])

df_eval_svm['Predicted'] = svm_preds
df_eval_lg['Predicted'] = lg_preds
df_eval_rf['Predicted'] = rf_preds